In [66]:
from friendlyfit.fitter import fit_events

In [68]:
fit_events(event_paths=['/Users/james/Downloads/astrocats/astrocats/supernovae/output/sne-2015-2019/SN2015bn.json'],
           model_paths=['example_model.json'],
           iterations=100)

ValueError: Tried to create an MPI pool, but there was only one MPI process available. Need at least two.